In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# from logitlens import LogitLens

In [ ]:
MODEL_NAME = "Tower-Babel/Babel-9B-Chat"
LANGUAGE = "English"
logit_lens = LogitLens(LANGUAGE, MODEL_NAME)
MODEL_NAME = MODEL_NAME.split("/")[-1]
path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}.csv"
df = pd.read_csv(path1)
df["splitted_tokens"] = df["splitted_tokens"].apply(ast.literal_eval)

In [ ]:
def setup_tokenizer(self):
    if self.tokenizer_name == "Tower-Babel/Babel-9B-Chat":
        self.tokenizer.add_special_tokens({'unk_token': 'UNK'})
        self.tokenizer.unk_token_id = self.tokenizer.convert_tokens_to_ids('UNK')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Tower-Babel/Babel-9B-Chat", use_fast=True)

In [ ]:
from transformers import AutoTokenizer

model_list = [
    "Tower-Babel/Babel-9B-Chat",
    "google/gemma-3-12b-it",
    "meta-llama/Llama-2-7b-chat-hf",
]
language_list = [
    "English",
    "German",
    # "Korean",
]
df_dict = {}
# MODEL_NAME = "Tower-Babel/Babel-9B-Chat"
# LANGUAGE = "English"
for MODEL_NAME in model_list:
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
        
    except:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, token=None)
    if tokenizer.unk_token is None:
        tokenizer.add_special_tokens({'unk_token': 'UNK'})
        tokenizer.unk_token_id = tokenizer.convert_tokens_to_ids('UNK')
    else:
        print(tokenizer.unk_token)
        

    for LANGUAGE in language_list:
        MODEL_NAME = MODEL_NAME.split("/")[-1]
        path1 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_v2.csv"
        df = pd.read_csv(path1)
        df["splitted_tokens"] = df["splitted_tokens"].apply(ast.literal_eval)
        splitted_input_tokens = []
        for tokens in df["splitted_tokens"]:
            ids = tokenizer.convert_tokens_to_ids(tokens)
            final_toknes = tokenizer.convert_ids_to_tokens(ids)
            splitted_input_tokens.append(final_toknes)
        df["splitted_input_tokens"] = splitted_input_tokens
        unk_token = tokenizer.unk_token
        df["include_UNK"] = df['splitted_input_tokens'].apply(lambda x: unk_token in x)
        df['num_UNK'] = df['splitted_input_tokens'].apply(lambda x: x.count(unk_token))
        df['proportion_UNK'] = df['splitted_input_tokens'].apply(lambda x: x.count(unk_token)/len(x))
        df_dict[(MODEL_NAME, LANGUAGE)] = df

In [ ]:
for k, df in df_dict.items():
    MODEL_NAME, LANGUAGE = k
    # path2 = f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/single_token_splitted_{MODEL_NAME}_{LANGUAGE}_with_UNK.csv"
    # v.to_csv(path2, index=False)
    # print(f"Saved {path2}")
    print(f"Model: {MODEL_NAME}, Language: {LANGUAGE}")
    print(df['include_UNK'].value_counts())

In [ ]:
df = df_dict[('gemma-3-12b-it', 'English')]
df

In [ ]:
import re
def normalize(word):
    return re.sub(r'\W+', '', word).lower()  # Remove non-alphanumeric chars and lowercase

def get_retrieval_rates(results_df):
    token_str_cols = [col for col in results_df.columns if col.endswith("_top_token_str")]
    retrieval_rates = [] 
    layers = []

    for col in token_str_cols:
        pred_word = results_df[col].apply(lambda x: [normalize(word) for word in x])
        original_word = results_df["word"].apply(normalize)
        retrieval_match = pred_word.combine(original_word, lambda preds, orig: orig in preds)
        retrieval_rate = retrieval_match.mean()
        retrieval_rates.append(retrieval_rate)
        # Extract layer number from column name, e.g., "layer_1_top1_token_str" -> 1
        layer_num = int(col.split("_")[1])
        layers.append(layer_num)

    # Sort by layer number
    sorted_indices = sorted(range(len(layers)), key=lambda i: layers[i])
    layers = [layers[i] for i in sorted_indices]
    retrieval_rates = [retrieval_rates[i] for i in sorted_indices]
    
    return layers, retrieval_rates

In [ ]:
df = df_dict[('Babel-9B-Chat', 'English')]
df_result = pd.read_csv("/home/hyujang/multilingual-inner-lexicon/output/RQ1/WordIdentity/single_token_simple_split_Babel-9B-Chat_English_v2.csv")
from ast import literal_eval

for col in df_result.columns:
    try:
        # Check if at least one value looks like a list
        if df_result[col].apply(lambda x: isinstance(x, str) and x.strip().startswith("[")).any():
            df_result[col] = df_result[col].apply(literal_eval)
    except (ValueError, SyntaxError):
        continue  # Skip columns that don't parse correctly
df_final = pd.concat([df, df_result.iloc[:,1:]], axis=1)

df_1 = df_final[~df_final.include_UNK]
layers, retrieval_rates_1 = get_retrieval_rates(results_df=df_1)

df_2 = df_final[df_final.include_UNK]
layers, retrieval_rates_2 = get_retrieval_rates(results_df=df_2)

plt.figure(figsize=(8, 5))
plt.plot(layers, retrieval_rates_1, marker='o', label='simple split - without UNK')
plt.plot(layers, retrieval_rates_2, marker='s', label='simple split - with UNK')
plt.xlabel("Layer")
plt.ylabel("Original Word Retrieval Rate (Top-3)")
plt.title(f"Top-3 Retrieval Rate per Layer ({"Babel-9B-Chat"}, {"English"})")
plt.ylim(0, 1)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
splitted_input_tokens = []
for tokens in df["splitted_tokens"]:
    ids = logit_lens.tokenizer.convert_tokens_to_ids(tokens)
    final_toknes = logit_lens.tokenizer.convert_ids_to_tokens(ids)
    splitted_input_tokens.append(final_toknes)
df["splitted_input_tokens"] = splitted_input_tokens
df["include_UNK"] = df['splitted_input_tokens'].apply(lambda x: "UNK" in x)
df['num_UNK'] = df['splitted_input_tokens'].apply(lambda x: x.count("UNK"))
df['proportion_UNK'] = df['splitted_input_tokens'].apply(lambda x: x.count("UNK")/len(x))

In [ ]:
df["proportion_UNK"].value_counts(normalize=True)